<a href="https://colab.research.google.com/github/TBFY/knowledge-graph-API/blob/master/UseCase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![TBFY-Banner](https://raw.githubusercontent.com/TBFY/general/master/figures/tbfy-banner.png)

# **USE CASE 1 - Organisations participating in similar contracting-processes**

This use case will show the possible relationship among organisations by comparising the contracting processes which they have played a role in.
First thing we are going to do is to get a list of organisations in the **core API**:

* **GET /organisation** that offers the whole list of organisations. No parameters are defined in this call that will display the following information:

|Field           |Description                                                        |Type   |Required|
|----------------|-------------------------------------------------------------------|-------|--------|
|**id**          |Corresponds to the identifier of the organisation, must be unique  |string |true    |

In [0]:
import requests
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

def RetrieveField(json,field):
  value=''
  try: 
    value = json[field]
  except:
    value=''
  return value

resp = requests.get('https://tbfy.librairy.linkeddata.es/kg-api/organisation')

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

organisation = [todo_item['id'] for todo_item in resp.json()]
repo_dict = {'Organisation': organisation}    
repo_df = pd.DataFrame(repo_dict)

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    repo_df = repo_df.stack().str.lstrip().unstack()
    repo_df = repo_df.style.set_properties(**{'text-align': 'left'})
repo_df

In the next step, we will obtain the list of contracting processes which the chosen organisation participates in. To do this, we will use the resource of the **core API**:

* **GET /organisation/{id}/contracting-process/** that offers the whole list of contracting processes which an organisation participates in. This resource will display the following information:

|Field           |Description                                                        |Type   |Required|
|----------------|-------------------------------------------------------------------|-------|--------|
|**id**          |Corresponds to the identifier of the contracting-process,          |string |true    |
|                |must be unique                                                     |       |        |

In [0]:
org1 = 'ocds-0c46vo-0001-a934dde3-e6c6-46bb-9c7b-80094a799d1e_Buyer'
resp = requests.get('https://tbfy.librairy.linkeddata.es/kg-api/organisation/' + org1 + '/contracting-process')

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
    
cp = [todo_item['id'] for todo_item in resp.json()]
repo_dict = {'Contracting Process': cp}    
repo_df = pd.DataFrame(repo_dict, index=[''])

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    repo_df = repo_df.stack().str.lstrip().unstack()
    repo_df = repo_df.style.set_properties(**{'text-align': 'left'})
repo_df

Then, the tenders involved in these contracting processes will be obtained. To do this, we will use the resource of the **core API**:

* **GET /contracting-process/{id}/tender/** that offers the whole list of contracting processes which an organisation participates in. This resource will display the following information:

|Field           |Description                                                        |Type   |Required|
|----------------|-------------------------------------------------------------------|-------|--------|
|**id**          |Corresponds to the identifier of the tender             ,          |string |true    |
|                |must be unique                                                     |       |        |


In [0]:
cp1 = 'ocds-0c46vo-0001-a934dde3-e6c6-46bb-9c7b-80094a799d1e_ContractingProcess'
resp = requests.get('https://tbfy.librairy.linkeddata.es/kg-api/contracting-process/' + cp1 + '/tender')

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

tender = [todo_item['id'] for todo_item in resp.json()]
repo_dict = {'Tender': tender}    
repo_df = pd.DataFrame(repo_dict, index=[''])

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    repo_df = repo_df.stack().str.lstrip().unstack()
    repo_df = repo_df.style.set_properties(**{'text-align': 'left'})
repo_df

Now, using the **search API**, a list of similar tender descriptions to the one obtained in the **core API** will be obtained:

In [0]:
td1 = 'ocds-0c46vo-0001-a934dde3-e6c6-46bb-9c7b-80094a799d1e_Tender_ocds-b5fd17-42de70ec-20c1-4bef-95f4-f0a48ceb662d-pd-437'
resp = requests.get('https://tbfy.librairy.linkeddata.es/search-api/documents/' + td1 + '/items?size=10&source=tender', auth=('oeg', 'oeg2018'))

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
#for todo_item in resp.json():
#    print('{} '.format(todo_item['id']))
    
tender = [todo_item['id'] for todo_item in resp.json()]
score = [todo_item['score'] for todo_item in resp.json()]
repo_dict = {'Tender': tender, 'Score': score}    
repo_df = pd.DataFrame(repo_dict)

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    repo_df = repo_df.stack().unstack()
    repo_df = repo_df.style.set_properties(**{'text-align': 'left'})
repo_df 

Once we have a list of similar tenders, we begin the way back to find out the participating organisations so, using **core API**, the list of contracting processes associated with that tender we will obtained using the resource:

* **GET /tender/{id}/contracting-process/** that offers the whole list of contracting processes related to a tender. This resource will display the following information:

|Field           |Description                                                        |Type   |Required|
|----------------|-------------------------------------------------------------------|-------|--------|
|**id**          |Corresponds to the identifier of the contracting-process,          |string |true    |
|                |must be unique                                                     |       |        |

In [0]:
td2 = 'ocds-0c46vo-0133-072076-2019_Tender_072076-2019_td'
resp = requests.get('https://tbfy.librairy.linkeddata.es/kg-api/tender/' + td2 + '/contracting-process')

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

cp = [todo_item['id'] for todo_item in resp.json()]
repo_dict = {'Contracting Process': cp}    
repo_df = pd.DataFrame(repo_dict, index=[''])

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    repo_df = repo_df.stack().str.lstrip().unstack()
    repo_df = repo_df.style.set_properties(**{'text-align': 'left'})
repo_df

Knowing the contracting processes, we can find out the organisations related to the contracting process using the resource:

* **GET /contracting-process/{id}/organisation/** that offers the whole list of organisations related to a contracting process. This resource will display the following information:

|Field           |Description                                                        |Type   |Required|
|----------------|-------------------------------------------------------------------|-------|--------|
|**id**          |Corresponds to the identifier of the organisation,                 |string |true    |
|                |must be unique                                                     |       |        |

In [0]:
cp2 = 'ocds-0c46vo-0133-072076-2019_ContractingProcess'
resp = requests.get('https://tbfy.librairy.linkeddata.es/kg-api/contracting-process/' + cp2 + '/organisation')

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

organisation = [todo_item['id'] for todo_item in resp.json()]
repo_dict = {'Organisation': organisation}    
repo_df = pd.DataFrame(repo_dict, index=[''])

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    repo_df = repo_df.stack().str.lstrip().unstack()
    repo_df = repo_df.style.set_properties(**{'text-align': 'left'})
repo_df

The last step is to make a comparision between both organisations. Basic information of organisations and the description of the tenders will be also provided.

In [0]:
org2 = 'ocds-0c46vo-0133-072076-2019_Buyer'

resp1 = requests.get('http://tbfy.librairy.linkeddata.es/kg-api/organisation/' + org1)
if resp1.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp1.status_code))

resp2 = requests.get('http://tbfy.librairy.linkeddata.es/kg-api/organisation/' + org2)
if resp2.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp1.status_code))

resp3 = requests.get('http://tbfy.librairy.linkeddata.es/kg-api/tender/' + td1)
if resp1.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp1.status_code))

resp4 = requests.get('http://tbfy.librairy.linkeddata.es/kg-api/tender/' + td2)
if resp2.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp2.status_code))


org = {'Organisation 1':[RetrieveField(resp1.json(),'id'), RetrieveField(resp1.json(),'name'), RetrieveField(resp1.json(),'streetAddress'), RetrieveField(resp1.json(),'locality'), RetrieveField(resp1.json(),'postalCode'), RetrieveField(resp1.json(),'country'), RetrieveField(resp1.json(),'contact'), RetrieveField(resp1.json(),'email'), RetrieveField(resp1.json(),'telephone'), RetrieveField(resp3.json(),'description')],
       'Comparision':['', '', '', '', '', '', '', '', '', ''],
       'Organisation 2':[RetrieveField(resp2.json(),'id'), RetrieveField(resp2.json(),'name'), RetrieveField(resp2.json(),'streetAddress'), RetrieveField(resp2.json(),'locality'), RetrieveField(resp2.json(),'postalCode'), RetrieveField(resp2.json(),'country'), RetrieveField(resp2.json(),'contact'), RetrieveField(resp2.json(),'email'), RetrieveField(resp2.json(),'telephone'), RetrieveField(resp4.json(),'description')]}
df = pd.DataFrame(org, index=['Identifier', 'Name', 'Address', 'Locality', 'Postal Code', 'Country', 'Contact', 'Email', 'Telephone', 'Tender Description'])

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df = df.stack().str.lstrip().unstack()
    df = df.style.set_properties(**{'text-align': 'justify'})
df
   